Item 48: Validate Subclasses with \_\_init_subclass\_\_

# You can't inherit two metaclasses

In [1]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if bases:
            if class_dict['sides']<3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class ValidateFilled(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if bases:
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return super().__new__(meta, name, bases, class_dict)

In [2]:
class Filled(metaclass=ValidateFilled):
    color = None
    
class Polygon(metaclass=ValidatePolygon):
    sides = None

In [3]:
class RedPentagon(Filled, Polygon):
    color = 'red'
    sides = 5

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

# Clumsy solution: complex hierarchy

In [4]:
class ValidatePolygon2(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['sides']<3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon2(metaclass=ValidatePolygon2):
    is_root = True
    sides = None # Must be specified by subclass

class ValidateFilledPolygon(ValidatePolygon2):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return super().__new__(meta, name, bases, class_dict)

class FilledPolygon(Polygon2, metaclass=ValidateFilledPolygon):
    is_root = True
    color = None # Must be specified by subclass

In [5]:
class GreenPentagon(FilledPolygon):
    color = 'green'
    sides = 5

greenie = GreenPentagon()
assert isinstance(greenie, Polygon2)

In [6]:
class OrangePentagon(FilledPolygon):
    color = 'orange'
    sides = 5

ValueError: Fill color must be supported

In [7]:
class RedLine(FilledPolygon):
    color = 'red'
    sides = 2

ValueError: Polygons need 3+ sides

# Smart solution using \_\_init_subclass__

In [8]:
class FilledZ:
    color = None # Must be specified by subclasses
    
    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.color not in ('red', 'green', 'blue'):
            raise ValueError('Fills need a valid color')

In [9]:
class PolygonZ:
    sides = None
    
    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides<3:
            raise ValueError('Polygons need 3+ sides')        
        

## Work example

In [10]:
class RedTriangle(FilledZ, PolygonZ):
    color = 'red'
    sides = 3

ruddy = RedTriangle()
assert isinstance(ruddy, FilledZ)
assert isinstance(ruddy, PolygonZ)

## Fail examples

In [11]:
print('Before class')

class BlueLine(FilledZ, PolygonZ):
    colof = 'blue'
    sides = 2

print('After class')

Before class


ValueError: Polygons need 3+ sides

In [12]:
print('Before class')

class BeigeSquare(FilledZ, PolygonZ):
    colof = 'beige'
    sides = 4

print('After class')

Before class


ValueError: Fills need a valid color

# Diamond inheritance

In [13]:
class Top:
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Top for {cls}')

class Left(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Left for {cls}')

class Right(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Right for {cls}')

class Bottom(Left, Right):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Bottom for {cls}')


Top for <class '__main__.Left'>
Top for <class '__main__.Right'>
Top for <class '__main__.Bottom'>
Right for <class '__main__.Bottom'>
Left for <class '__main__.Bottom'>
